In [1]:
import dask
import xgboost as xgb
from Train_test_days_split import *

In [3]:
xgb.__version__

'1.0.0'

In [4]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
def get_slurm_dask_client(n_workers, n_cores):
    cluster = SLURMCluster(cores=n_cores,
                           memory='32GB',
                           project="co_aiolos",
                           walltime="02:00:00",
                           queue="savio2_gpu",
                           job_extra=['--gres=gpu:1','--cpus-per-task=2'])

    cluster.scale(n_workers)
    client = Client(cluster)
    return client
def get_slurm_dask_client_bigmem(n_nodes):
    cluster = SLURMCluster(cores=24,
                           memory='128GB',
                           project="co_aiolos",
                           walltime="02:00:00",
                           queue="savio2_bigmem",
                           job_extra=['--qos="savio_lowprio"'])

    cluster.scale(n_nodes*6)
    client = Client(cluster)
    return client
def get_slurm_dask_client_savio2(n_nodes):
    cluster = SLURMCluster(cores=12,
                           memory='64GB',
                           project="co_aiolos",
                           walltime="12:00:00",
                           queue="savio",
                           job_extra=['--qos="aiolos_savio_normal"'])

    cluster.scale(n_nodes*6)
    client = Client(cluster)
    return client
def get_slurm_dask_client_savio3(n_nodes):
    cluster = SLURMCluster(cores=32,
                           memory='96GB',
                           project="co_aiolos",
                           walltime="72:00:00",
                           queue="savio3",
                           job_extra=['--qos="aiolos_savio3_normal"'])

    cluster.scale(n_nodes*16)
    client = Client(cluster)
    return client

In [5]:
client = get_slurm_dask_client_savio3(3)

In [87]:
client

Client Scheduler: tcp://10.0.0.24:39571 Dashboard: http://10.0.0.24:8787/status,Cluster Workers: 40 Cores: 160 Memory: 480.00 GB


In [88]:
scheduler_info = client.scheduler_info()
workers = client.get_worker_logs()
list_of_workers = list(workers.keys())
list_of_workers

['tcp://10.0.7.109:36046',
 'tcp://10.0.7.109:36489',
 'tcp://10.0.7.109:38404',
 'tcp://10.0.7.109:39345',
 'tcp://10.0.7.109:39416',
 'tcp://10.0.7.109:43160',
 'tcp://10.0.7.109:44460',
 'tcp://10.0.7.19:33501',
 'tcp://10.0.7.19:35369',
 'tcp://10.0.7.19:36658',
 'tcp://10.0.7.19:38046',
 'tcp://10.0.7.19:38555',
 'tcp://10.0.7.19:39284',
 'tcp://10.0.7.19:42947',
 'tcp://10.0.7.19:45250',
 'tcp://10.0.7.22:34885',
 'tcp://10.0.7.22:36736',
 'tcp://10.0.7.22:39354',
 'tcp://10.0.7.22:39981',
 'tcp://10.0.7.22:40237',
 'tcp://10.0.7.22:40608',
 'tcp://10.0.7.22:41927',
 'tcp://10.0.7.22:46032',
 'tcp://10.0.7.23:35310',
 'tcp://10.0.7.24:33374',
 'tcp://10.0.7.24:33659',
 'tcp://10.0.7.24:34205',
 'tcp://10.0.7.24:35194',
 'tcp://10.0.7.24:38574',
 'tcp://10.0.7.24:40438',
 'tcp://10.0.7.24:41938',
 'tcp://10.0.7.24:44382',
 'tcp://10.0.7.26:36224',
 'tcp://10.0.7.26:37187',
 'tcp://10.0.7.26:37521',
 'tcp://10.0.7.26:40172',
 'tcp://10.0.7.26:42891',
 'tcp://10.0.7.26:43778',
 'tcp

In [ ]:
#1.scheduler: tasks --> task graph 
#2.worker nodes

1. false dependence
b = a[i]+c
d = a[i+1]+c
compiler: b and d has to be processed in the same core with a saved
total_datasets = dict()
total_datasets['no2'] = no2 from ncdf
total_datasets['pres'] = pres from ncdf
each worker need memory to save total_datasets (full matrix)


2. nc.Datasets
data = read all data from ncdf file (~5G)
data['no2'] --> numpy array --> dask array (slow!!!)
package: xarray read in data from a ncdf file to a dask array (fast)

3. Solution
for each feature, call xarray, save the target feature data, and then do processing.



In [9]:
import xarray as xr
import time
orig_filenames = sorted(glob(os.path.join(orig_file_path, 'met_conus*')))
orig_filenames = orig_filenames[0:5]
train_filenames, test_filenames = train_test_filename(orig_filenames)
filename = train_filenames[0]

In [93]:
nvel = 29
ngrid = 100250
ntime = 16
keep_indx = np.load('target_cells_index.npy')
reader = csv.reader(open("surrouding_cells.csv", "r"))
surr_arr = []
for row in reader:
    this_arr = list(map(int, row[1].strip(']|[').split(',')))
    surr_arr.append(this_arr)
surr_arr = np.array(surr_arr)[keep_indx, :]
def read_var_from_file_xr(filename, varname):
    ds = xr.open_mfdataset(filename, parallel=True)
    this_var = da.array(ds[varname][1:, keep_indx, :]).flatten()
    return this_var
def read_var_from_file_nc(filename, varname):
    ds = nc.Dataset(filename)
    this_var = da.array(ds[varname][1:, keep_indx, :]).flatten()
    return this_var
def read_2d_var_from_file_xr(filename, varname):
    ds = xr.open_mfdataset(filename, parallel=True)
    this_var = da.repeat(da.array(ds[varname][1:, keep_indx, :]),nvel,axis=2).flatten()
    return this_var
def read_wind_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    stg_u = da.array(ds['U'])
    stg_v = da.array(ds['V'])
    stg_w = da.array(ds['W'][1:, keep_indx, :])
    u_indx_left, u_indx_right, v_indx_bot, v_indx_up = find_indx_for_wind()
    wind_u = (stg_u[1:, u_indx_left, :] + stg_u[1:, u_indx_right, :])/2
    u = wind_u[:, keep_indx, :].flatten()
    wind_v = (stg_v[1:, v_indx_up, :] + stg_v[1:, v_indx_bot, :])/2
    v = wind_v[:, keep_indx, :].flatten()
    w = (stg_w[:, :, 1:] + stg_w[:, :, :-1])/2
    w = w.flatten()
    return da.stack([u,v,w])
def read_lightning_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    cum_ic_flash = da.array(ds['IC_FLASHCOUNT'][:])
    cum_cg_flash = da.array(ds['CG_FLASHCOUNT'][:])
    ic_flash = da.repeat(cum_ic_flash[1:, :, :]-cum_ic_flash[:-1, :, :], nvel, axis=2)
    cg_flash = da.repeat(cum_cg_flash[1:, :, :]-cum_cg_flash[:-1, :, :], nvel, axis=2)
    e_lightning = ic_flash + cg_flash
    this_var = e_lightning[:, keep_indx, :].flatten()
    surrs = []
    surrs.append(this_var)
    for i in range(0, 24):
        surr_indx = surr_arr[:, i]
        surrs.append(e_lightning[:, surr_indx, :].flatten())
    return da.stack(surrs)
def read_anthro_emis_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    e_no_lower = da.array(ds['E_NO'])[1:, :, :]
    e_no_upper = da.zeros((ntime, e_no_lower.shape[1], nvel - e_no_lower.shape[2]))
    e_no = da.concatenate([e_no_lower, e_no_upper], axis=2)
    e_no_total = []
    e_no_total.append(e_no[:, keep_indx, :].flatten())
    for i in range(0, 24):
        surr_indx = surr_arr[:, i]
        e_no_total.append(e_no[:, surr_indx, :].flatten())
    return da.stack(e_no_total)

In [90]:
varname = 'no2'
future = client.submit(read_var_from_file_xr, filename, varname)

In [94]:
varname = 'no2'
future = client.submit(read_var_from_file_nc, filename, varname)


<Future: pending, key: read_var_from_file_nc-321b73818f2009dde2e19260483e9d9f>

In [98]:
varnames = ['no2','pres','temp','CLDFRA']
futures = []
for varname in varnames:
    future = client.submit(read_var_from_file_xr, filename, varname)
    futures.append(future)
dirt_feature = da.stack([future.result() for future in futures])
varnames_2d = ['COSZEN', 'PBLH', 'LAI', 'HGT', 'SWDOWN', 'GLW']
futures_2d = []
for varname in varnames_2d:
    future_2d = client.submit(read_2d_var_from_file_xr, filename, varname)
    futures_2d.append(future_2d)
dirt2d_feature = da.stack([future.result() for future in futures_2d])    
wind_feature = client.submit(read_wind_from_file_xr, filename)
anthroemis_future = client.submit(read_anthro_emis_from_file_xr, filename)
lightning_future = client.submit(read_lightning_from_file_xr, filename)

In [99]:
lightning_future

<Future: finished, type: dask.Array, key: read_lightning_from_file_xr-7db92c37e3bea698608b94606ec5d4f0>

In [103]:
datasets = da.transpose(da.concatenate((dirt_feature, dirt2d_feature, wind_feature.result(), anthroemis_future.result(), lightning_future.result()),axis=0))
X = datasets[:, 1:]
y = datasets[:, 0]
X,y = dask.persist(X,y)
del datasets

In [104]:
for future in futures:
    del future
for future in futures_2d:
    del future
del wind_feature
del anthroemis_future
del lightning_future

dask.array<getitem, shape=(46516000,), dtype=float64, chunksize=(23258000,), chunktype=numpy.ndarray>

In [105]:
X = X.rechunk(chunks=(390000,62))
y = y.rechunk(chunks=(390000,1))

In [86]:
X

dask.array<rechunk-merge, shape=(46516000, 62), dtype=float64, chunksize=(390000, 62), chunktype=numpy.ndarray>

In [83]:
y.rechunk(chunks=(390000,1))


dask.array<rechunk-merge, shape=(46516000,), dtype=float64, chunksize=(390000,), chunktype=numpy.ndarray>

In [84]:
dtrain = xgb.dask.DaskDMatrix(client, X, y)

CancelledError: 

In [ ]:
output = xgb.dask.train(client,
                        {'verbosity': 2,
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])

In [75]:
client

Client Scheduler: tcp://10.0.0.24:39571 Dashboard: http://10.0.0.24:8787/status,Cluster Workers: 40 Cores: 160 Memory: 480.00 GB


In [40]:
#create dtrain for xgboost
import dask_xgboost
params = {'objective': 'binary:logistic',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5}
bst = dask_xgboost.train(client, params, X, y, num_boost_round=10)

#datasets_future = client(client.scatter(datasets))
#dtrain_future = client.submit
#dtrain_future = client.submit(xgb.dask.DaskDMatrix, X, y)

ModuleNotFoundError: No module named 'dask_xgboost'

In [ ]:
import xgboost as xgb
def create_dtrain(datasets):
    X = datasets[:, 1,:]
    y = datasets[:, 0]
    dtrain = xgb.dask.DaskDMatrix(client, X, y)
    return dtrain

In [216]:
dtrain_future.result()

TypeError: Expecting <class 'distributed.client.Client'> or <class 'NoneType'>.  Got <class 'dask.array.core.Array'>

In [177]:
anthroemis_future.result()

dask.array<stack, shape=(25, 46516000), dtype=float64, chunksize=(1, 23258000), chunktype=numpy.ndarray>

In [168]:
varnames_2d = ['COSZEN', 'PBLH', 'LAI', 'HGT', 'SWDOWN', 'GLW']
futures_2d = []
for varname in varnames_2d:
    future_2d = client.submit(read_2d_var_from_file_xr, filename, varname)
    futures_2d.append(future_2d)
future_2d

<Future: pending, key: read_2d_var_from_file_xr-74b0c2381f36b9344e1ac5f6dd365148>

In [170]:
futures_2d[0].result()

dask.array<reshape, shape=(46516000,), dtype=float64, chunksize=(46516000,), chunktype=numpy.ndarray>

In [155]:

    
lightning_future = client.submit(read_lightning_from_file_xr, filename)

In [156]:
futures[-1].result()

dask.array<reshape, shape=(46516000,), dtype=float64, chunksize=(46516000,), chunktype=numpy.ndarray>

In [163]:
feature_v1 = da.transpose(da.stack([future.result() for future in futures]))
np.concatenate((feature_v1,lightning_future.result()),axis=1)

dask.array<concatenate, shape=(46516000, 30), dtype=float64, chunksize=(46516000, 1), chunktype=numpy.ndarray>

In [86]:
feature = client.submit(concatenate_features, futures)

In [88]:
feature.result()

AttributeError: 'Array' object has no attribute 'result'

In [77]:
features

dask.array<transpose, shape=(46516000, 4), dtype=float64, chunksize=(11629000, 1), chunktype=numpy.ndarray>

In [76]:
for future in futures:
    del future

In [29]:
#to do: surrounding lightning and vertical distribution of lightnings haven't been implemented yet
import time
orig_filenames = sorted(glob(os.path.join(orig_file_path, 'met_conus*')))
orig_filenames = orig_filenames[0:5]
train_filenames, test_filenames = train_test_filename(orig_filenames)
x_total = []
y_total = []
add_total= []
starttime = time.time()
for train_filename in train_filenames:
    print('Reading in data from {}'.format(train_filename))
    additional_arr, x_arr, y_arr, x_labels, additional_features = read_orig_file_from_wrf(train_filename)
    add_total.append(additional_arr)
    x_total.append(x_arr)
    y_total.append(y_arr)
print("--- {} minutes ---".format((time.time() - starttime)/60))

Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_002
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_003
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_005
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_004
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_001
--- 0.25605957508087157 minutes ---


In [ ]:
future = client.compute(x_total)
client.close()

In [ ]:
future[0].result()

In [ ]:
#prepare matrix for ml
import xgboost as xgb
starttime = time.time()
dtrain = xgb.dask.DaskDMatrix(client, da.array(x_total), da.array(y_total))
print("--- {} minutes ---".format((time.time() - starttime)/60))

In [ ]:
client()

In [5]:
#create a file saving the index of target cells
create_index_list_for_target_cells()

In [3]:
#create a file saving the indexes of surrounding emissions
create_index_dict_for_surrouding_emis()

Make surrouding cells for cell lat 0
Make surrouding cells for cell lat 1
Make surrouding cells for cell lat 2
Make surrouding cells for cell lat 3
Make surrouding cells for cell lat 4
Make surrouding cells for cell lat 5
Make surrouding cells for cell lat 6
Make surrouding cells for cell lat 7
Make surrouding cells for cell lat 8
Make surrouding cells for cell lat 9
Make surrouding cells for cell lat 10
Make surrouding cells for cell lat 11
Make surrouding cells for cell lat 12
Make surrouding cells for cell lat 13
Make surrouding cells for cell lat 14
Make surrouding cells for cell lat 15
Make surrouding cells for cell lat 16
Make surrouding cells for cell lat 17
Make surrouding cells for cell lat 18
Make surrouding cells for cell lat 19
Make surrouding cells for cell lat 20
Make surrouding cells for cell lat 21
Make surrouding cells for cell lat 22
Make surrouding cells for cell lat 23
Make surrouding cells for cell lat 24
Make surrouding cells for cell lat 25
Make surrouding cells 

Make surrouding cells for cell lat 216
Make surrouding cells for cell lat 217
Make surrouding cells for cell lat 218
Make surrouding cells for cell lat 219
Make surrouding cells for cell lat 220
Make surrouding cells for cell lat 221
Make surrouding cells for cell lat 222
Make surrouding cells for cell lat 223
Make surrouding cells for cell lat 224
Make surrouding cells for cell lat 225
Make surrouding cells for cell lat 226
Make surrouding cells for cell lat 227
Make surrouding cells for cell lat 228
Make surrouding cells for cell lat 229
Make surrouding cells for cell lat 230
Make surrouding cells for cell lat 231
Make surrouding cells for cell lat 232
Make surrouding cells for cell lat 233
Make surrouding cells for cell lat 234
Make surrouding cells for cell lat 235
Make surrouding cells for cell lat 236
Make surrouding cells for cell lat 237
Make surrouding cells for cell lat 238
Make surrouding cells for cell lat 239
Make surrouding cells for cell lat 240
Make surrouding cells for

In [4]:
print('additional features: {}'.format(additional_features))
print('features: {}'.format(x_labels))

additional features: ['xlon', 'xlat', 'date', 'elev', 'hour', 'IC_FLASHCOUNT', 'CG_FLASHCOUNT']
features: ['COSZEN', 'U', 'V', 'W', 'PBLH', 'E_NO', 'pres', 'temp', 'LAI', 'HGT', 'CLDFRA', 'SWDOWN', 'GLW', 'anthro_surr_emis_00', 'lightning_surr_emis_00', 'anthro_surr_emis_01', 'lightning_surr_emis_01', 'anthro_surr_emis_02', 'lightning_surr_emis_02', 'anthro_surr_emis_03', 'lightning_surr_emis_03', 'anthro_surr_emis_04', 'lightning_surr_emis_04', 'anthro_surr_emis_05', 'lightning_surr_emis_05', 'anthro_surr_emis_06', 'lightning_surr_emis_06', 'anthro_surr_emis_07', 'lightning_surr_emis_07', 'anthro_surr_emis_08', 'lightning_surr_emis_08', 'anthro_surr_emis_09', 'lightning_surr_emis_09', 'anthro_surr_emis_10', 'lightning_surr_emis_10', 'anthro_surr_emis_11', 'lightning_surr_emis_11', 'anthro_surr_emis_12', 'lightning_surr_emis_12', 'anthro_surr_emis_13', 'lightning_surr_emis_13', 'anthro_surr_emis_14', 'lightning_surr_emis_14', 'anthro_surr_emis_15', 'lightning_surr_emis_15', 'anthro_sur

In [5]:
x_total[0][0]

dask.array<reshape, shape=(46516000,), dtype=float32, chunksize=(46516000,), chunktype=numpy.MaskedArray>

In [6]:
client = Client()

In [7]:
client

Client Scheduler: tcp://127.0.0.1:61441 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [12]:
#save_arr = da.array(save_arr.T)
client = get_slurm_dask_client(1, 1)

/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44606 instead
  http_address["port"], self.http_server.port


In [15]:
client

Client Scheduler: tcp://10.0.0.24:42736 Dashboard: http://10.0.0.24:44606/status,Cluster Workers: 1 Cores: 1 Memory: 32.00 GB


In [ ]:
#fit ml model
starttime = time.time()
output = xgb.dask.train(client,
                        {'verbosity': 2,
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])
print("--- {} minutes ---".format((time.time() - starttime)/60))

In [34]:
cluster = SLURMCluster(cores=24, memory = '128 GB', project='co_aiolos',queue='savio2_bigmem',walltime='02:00:00')

/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36835 instead
  http_address["port"], self.http_server.port


In [35]:
cluster.scale(1)

In [36]:
client = Client(cluster)

In [38]:
client

Client Scheduler: tcp://10.0.0.24:43513 Dashboard: http://10.0.0.24:36835/status,Cluster Workers: 0 Cores: 0 Memory: 0 B
